# Robustness of founded policies

1) Regret
2) satisfying
3) signal noise_to_ratio

In [68]:
# Import general python packages
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over,time_step_0,time_step_1, time_step_2, time_step_3, time_step_4
from sklearn.cluster import KMeans

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome, TimeSeriesOutcome, ArrayOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results, Policy
from ema_workbench.em_framework.optimization import (EpsilonProgress)
from ema_workbench.analysis import parcoords
from itertools import cycle

### Initializing model

In [37]:

def initialize_model():
    ema_logging.log_to_stderr(ema_logging.INFO)
    print("Initializing model...")
    dike_model, planning_steps = get_model_for_problem_formulation(7)
    print("Model initialized.")
    return dike_model, planning_steps

# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor, uncertainties, levers):
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE

    model.uncertainties = uncertainties
    model.levers = levers

    cost_variables = []
    cost_variables.extend(
    [
        f"{dike}_{e}"
        for e in ["Expected Annual Damage", "Dike Investment Costs"]
        for dike in function.dikelist
    ])
    cost_variables.extend([f"RfR Total Costs"])
    cost_variables.extend([f"Expected Evacuation Costs"])

    if problem_formulation_actor == 6:  # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome(f'Total_period_Costs_0',
                          variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
                          function=time_step_0, kind=direction),
            ScalarOutcome(f'Total_period_Costs_1',
                          variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
                          function=time_step_1, kind=direction),
            # ScalarOutcome(f'Total_period_Costs_2',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_2, kind=direction),
            # ScalarOutcome(f'Total_period_Costs_3',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_3, kind=direction),
            # ScalarOutcome(f'Total_period_Costs_4',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_4, kind=direction),
            ScalarOutcome('Expected Annual Damage A1_', variable_name='A.1_Expected Annual Damage', function=sum_over,
                          kind=direction),
            ScalarOutcome('Expected Annual Damage A2_', variable_name='A.2_Expected Annual Damage', function=sum_over,
                          kind=direction),
            ScalarOutcome('Expected Annual Damage A3_', variable_name='A.3_Expected Annual Damage', function=sum_over,
                          kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function=sum_over, kind=direction),
            ScalarOutcome("Expected Number of Deaths_", variable_name=
            [f"{dike}_Expected Number of Deaths" for dike in function.dikelist], function=sum_over, kind=direction)]


    elif problem_formulation_actor == 7:  # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome(f'Total_period_Costs_0',
                          variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
                          function=time_step_0, kind=direction),
            ScalarOutcome(f'Total_period_Costs_1',
                          variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
                          function=time_step_1, kind=direction),
            # ScalarOutcome(f'Total_period_Costs_2',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_2, kind=direction),
            # ScalarOutcome(f'Total_period_Costs_3',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_3, kind=direction),
            # # ScalarOutcome(f'Total_period_Costs_4',
            #               variable_name=dike_model.outcomes['Total_period_Costs'].variable_name,
            #               function=time_step_4, kind=direction),
            ScalarOutcome('Expected Annual Damage A4_', variable_name='A.4_Expected Annual Damage', function=sum_over,
                          kind=direction),
            ScalarOutcome('Expected Annual Damage A5_', variable_name='A.5_Expected Annual Damage', function=sum_over,
                          kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function=sum_over, kind=direction),
            ScalarOutcome("Expected Number of Deaths_", variable_name=
            [f"{dike}_Expected Number of Deaths" for dike in function.dikelist], function=sum_over, kind=direction)]

    else:
        raise TypeError('unknown identifier')
    return model

### Overijssel
if __name__ == '__main__':
    dike_model, planning_steps = initialize_model()

    uncertainties = dike_model.uncertainties
    levers = dike_model.levers
    
    model = problem_formulation_actor(7, uncertainties, levers)

    # Deepcopying the uncertainties and levers
    uncertainties = copy.deepcopy(dike_model.uncertainties)
    levers = copy.deepcopy(dike_model.levers)

    # Running the optimization for Overijssel
    function = DikeNetwork()

In [69]:
policy_set= pd.read_csv("Overijssel Multi MORDM_Policies.csv")

policy_set

policy_snip=[]
for i in range (2, 8):
    policy_snip.append(policy_set.iloc[:,-i].idxmin())
    policy_snip.append(policy_set.iloc[:,-i].idxmax())
    #print(policy_set.columns[-i])

len(policy_snip)

pareto_df= policy_set

#Selecteer de kolommen met (objectieven)
objective_columns = pareto_df.columns[-7:-1]
objective_columns

#  partition objectives columns in 3 segments and select one solution per segment
selected_policies = pd.DataFrame()

for objective in objective_columns:
    # Sort for based on Pareto outcome
    pareto_df_sorted = pareto_df.sort_values(by=objective)

    # split in 3 segments
    indices = np.linspace(0, len(pareto_df_sorted) - 1, 4, dtype=int)

    # set to list of indices
    selected_indices = (indices[:-1] + np.diff(indices) // 2).tolist()

    # Select rows and add to policies
    selected_policies = pd.concat([selected_policies, pareto_df_sorted.iloc[selected_indices]])

policy_snip2=  selected_policies.index.tolist()
policy_snip2

total_snip= policy_snip + policy_snip2
len(total_snip) #30 = 12+18 so perfect
unique_snip = list(set(total_snip))
len(unique_snip)

policies = policy_set.loc[unique_snip]
policies = policies.iloc[:,1:51]


rcase_policies = []
for i, policy in policies.iterrows():
    rcase_policies.append(Policy(str(i), **policy.to_dict()))

n_scenarios = 1000
with MultiprocessingEvaluator(model) as evaluator:
    reference_policies_results = evaluator.perform_experiments(n_scenarios,
                                            rcase_policies)
save_results(reference_policies_results, 'MultiMORDM_O.tar.gz')

### Loading in the datasets

In [82]:
experiments, outcomes = load_results('MultiMORDM_O.tar.gz')

df_exp = pd.DataFrame.from_dict(experiments)
df_out = pd.DataFrame.from_dict(outcomes)

In [83]:
experiments

In [84]:
for _ in outcomes:
    print(_)

In [85]:
policies = experiments.iloc[:,21:]
policies

In [86]:
df_out

### 1. Regret

In [87]:
import pandas as pd
import numpy as np

def calculate_regret(data, best):
    return np.abs(best-data)

worstcase_regret = {}
worstcase_max_regret = {}

for outcome in model.outcomes:
    policy_column = experiments['policy']
    
    # Create a DataFrame with all the relevant information
    data = pd.DataFrame({
        outcome.name: outcomes[outcome.name], 
        "policy": experiments['policy'],
        "scenario_id": experiments['scenario']
    })
    
    # Reorient the data by indexing with policy and scenario_id
    data = data.pivot(index='scenario_id', columns='policy')
    
    # Flatten the resulting hierarchical index resulting from pivoting
    data.columns = data.columns.get_level_values(1)
   
    # Control the broadcasting by converting to NumPy arrays
    max_values = data.max(axis=1).to_numpy()[:, np.newaxis]
    data_values = data.to_numpy()
    
    # Calculate the outcome regret
    outcome_regret = np.abs(max_values - data_values)
    
    # Store the results
    worstcase_regret[outcome.name] = outcome_regret
    worstcase_max_regret[outcome.name] = outcome_regret.max(axis=0)


In [88]:
worstcase_max_regret = pd.DataFrame(worstcase_max_regret)
sns.heatmap(worstcase_max_regret/worstcase_max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [89]:
colors = sns.color_palette()

data = worstcase_max_regret

# makes it easier to identify the policy associated with each line
#in the parcoords plot
#data['policy'] = data.index.astype("float64")

limits = parcoords.get_limits(data)
limits.loc[0, ['Total_period_Costs_0', 'Total_period_Costs_1','Expected Annual Damage A4_', 'Expected Annual Damage A5_', 'Total Costs', 'Expected Number of Deaths_']] = 0

palettes = [sns.color_palette(palette, 20) for palette in ['tab20', 'tab20b', 'tab20c']]
combined_palette = cycle([color for palette in palettes for color in palette])
colors = [next(combined_palette) for _ in range(35)]


paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()
    
plt.show()

In [90]:
#no need for distribution as max regret is what we are intrested in especially with deaths and damages.. MAX regret is how good it could have been in a certain scenario if you would have chosen another policy

### 2. Signal to Noise

In [91]:
def signalnoise(data, direction):
    mean = np.mean(data)
    std = np.std(data)
    
    #We minimize all objectives therefore a lower mean and lower std is wanted, so we want the outcome to be as small as possible
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [92]:
overall_scores = {}
for policy in np.unique(experiments['policy']):
    scores = {}
    
    logical = experiments['policy']==policy
    
    for outcome in model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = signalnoise(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
scores = pd.DataFrame.from_dict(overall_scores).T
scores

In [93]:
from ema_workbench.analysis import parcoords

data = scores
limits = parcoords.get_limits(data)
limits.loc[0, ['Total_period_Costs_0', 'Total_period_Costs_1','Expected Annual Damage A4_', 'Expected Annual Damage A5_', 'Total Costs', 'Expected Number of Deaths_']] = 0

paraxes = parcoords.ParallelAxes(limits)

for i, (policy, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(i), color=colors[i])
paraxes.legend()
#paraxes.invert_axis('max_P')
plt.show()

In [94]:
#This shows the Signal to Noice, which is expected * standard deviation for our minimization. a lower score is better. This is about over different scenarios the average and standard deviation, punishing for too much deviation over the scenarios.

##  3. Satisficing

In [95]:

thresholds = {'Total_period_Costs_0':2500000000, 'Total_period_Costs_1':2000000000, 'Expected Annual Damage A4_': 2000000, 'Expected Annual Damage A5_': 2000000, 'Total Costs': 7000000000, 'Expected Number of Deaths_': 1}


overall_scores = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    scores = {}
    for k, v in outcomes.items():
        try:
            n = np.sum(v[logical]>=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/100 #set to number of scenarios
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)

for i, (policy, row) in enumerate(overall_scores.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(i), color=colors[i])
paraxes.legend()
plt.show()

In [ ]:
#hoeveel procent van de scenarios gaat een policy over the treshold heen van een bepaalde objective? is heirbovenhet antwoord 